In [50]:
import pickle
import numpy as np
import torch
from torch import nn
import pytorch_lightning as pl

In [70]:
class Net(nn.Module):
    """
    Network
    """
    def __init__(self, in_feat, out_feat, hidden_layers: list[int]):
        super(Net, self).__init__()

        hidden_layers.append(out_feat)

        list_seq = [
            nn.Linear(
                in_feat,
                hidden_layers[0]
            ),
            nn.ReLU()
        ]
        for i, num in enumerate(hidden_layers[1:]):
            list_seq.append(
                nn.Linear(
                    hidden_layers[i],
                    num
                )
            )
            list_seq.append(nn.LeakyReLU())

        # list_seq.append(nn.LogSoftmax(-1))
        
        self.layers = nn.Sequential(
            *list_seq
        )

    def forward(self, x):
        return self.layers(x)


class NetModel(pl.LightningModule):

    def __init__(
        self, lr=0.001, warmup=10, max_iters=300, hidden_layers='4,7'
    ):
        super().__init__()
        self.save_hyperparameters()

        hidden_layers = list(map(int, hidden_layers.split(',')))
        self.net = Net(4, 3, hidden_layers)
        self.example_input_array = torch.zeros(1, 4).type('torch.FloatTensor')

    def forward(self, x):
        return self.net(x)

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)
        lr_scheduler = LogWarmupScheduler(
            optimizer,
            warmup=self.hparams.warmup,
            max_iters=self.hparams.max_iters
        )
        return [optimizer], [{'scheduler': lr_scheduler, 'interval': 'step'}]

    def training_step(self, batch, batch_idx):
        x, y = batch
        
        out = self.forward(x)
                
        loss = F.nll_loss(out, y)
        
        self.log('loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        
        out = self.forward(x)
        
        loss = F.nll_loss(out, y)

        self.log('val_loss', loss)

In [5]:
with open("iris.pkl", "rb") as fio:
    iris = pickle.load(fio)

In [7]:
x = iris["data"]
input_shape = (4,)

In [54]:
# prepare output data
target = iris["target"]
num_classes = 3

In [71]:
net = NetModel.load_from_checkpoint("ckpts/lightning_logs/version_0/checkpoints/epoch=117-step=1652.ckpt")

In [72]:
ii = 59
inp = torch.tensor(x[ii]).type("torch.FloatTensor").expand(1, -1)
print(net.forward(inp), target[ii])

tensor([[-0.0343,  6.2539,  3.0818]], grad_fn=<LeakyReluBackward0>) 1


In [44]:
len([p for p in net.Parameters()])

AttributeError: 'Net' object has no attribute 'Parameters'

In [59]:
with open("partition.pkl", "rb") as fio:
    partition = pickle.load(fio)

In [64]:
[target[i] for i in partition['test']]

[1, 0, 2, 0, 0, 2, 1, 1, 1, 0, 0, 0, 1, 1, 1]

In [66]:
net.parameters()

<generator object Module.parameters at 0x7f3dd97d87b0>

In [87]:
for ii in partition['test']:
    inp = torch.tensor(x[ii]).type("torch.FloatTensor").expand(1, -1)
    print(x[ii], net.forward(inp), target[ii])

[5.6 3.  4.1 1.3] tensor([[-0.0332,  7.0388,  2.6194]], grad_fn=<LeakyReluBackward0>) 1
[4.8 3.4 1.9 0.2] tensor([[ 7.3161e+00,  1.3854e+00, -6.8920e-03]], grad_fn=<LeakyReluBackward0>) 0
[6.2 3.4 5.4 2.3] tensor([[-0.0651,  3.0327,  9.0519]], grad_fn=<LeakyReluBackward0>) 2
[5.2 4.1 1.5 0.1] tensor([[10.8327,  0.1262, -0.0153]], grad_fn=<LeakyReluBackward0>) 0
[5.5 4.2 1.4 0.2] tensor([[11.2276,  0.0900, -0.0158]], grad_fn=<LeakyReluBackward0>) 0
[7.7 3.8 6.7 2.2] tensor([[-0.0753,  4.3959, 10.1352]], grad_fn=<LeakyReluBackward0>) 2
[6.1 2.9 4.7 1.4] tensor([[-0.0423,  6.6899,  4.1530]], grad_fn=<LeakyReluBackward0>) 1
[6.  2.9 4.5 1.5] tensor([[-0.0413,  6.5720,  4.0461]], grad_fn=<LeakyReluBackward0>) 1
[6.4 2.9 4.3 1.3] tensor([[-0.0356,  7.5317,  2.8239]], grad_fn=<LeakyReluBackward0>) 1
[5.5 3.5 1.3 0.2] tensor([[10.0876,  0.3859, -0.0135]], grad_fn=<LeakyReluBackward0>) 0
[5.2 3.5 1.5 0.2] tensor([[ 9.1751,  0.6907, -0.0114]], grad_fn=<LeakyReluBackward0>) 0
[4.6 3.2 1.4 0.2] te

In [92]:
try:
    raise RuntimeError("nope")
except Exception as e:
    print(e.__str__())

nope


In [93]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  